<a href="https://colab.research.google.com/github/Ken79151/-113B-/blob/main/EX04_03_%E5%AE%A2%E6%88%B6%E5%88%86%E7%BE%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 資料收集

In [2]:
!wget -O car_models.csv https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
# 載入模組
import pandas as pd

# 讀取網路上的 CSV 檔
import pandas as pd
url = 'https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv'
df = pd.read_csv(url)
df.head()

--2025-06-03 06:40:37--  https://raw.githubusercontent.com/imchihchao/aop113b/main/materials/04-customer.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2839 (2.8K) [text/plain]
Saving to: ‘car_models.csv’

car_models.csv      100%[===================>]   2.77K  --.-KB/s    in 0s      

2025-06-03 06:40:38 (40.0 MB/s) - ‘car_models.csv’ saved [2839/2839]



,性別,年齡,收入（千）,消費指數（1~100）
0,女,74,38,81
1,女,51,71,91
2,女,30,65,10
3,女,88,49,17
4,女,55,48,70


# 資料清理

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   性別           200 non-null    object
 1   年齡           200 non-null    int64 
 2   收入（千）        200 non-null    int64 
 3   消費指數（1~100）  200 non-null    int64 
dtypes: int64(3), object(1)
memory usage: 6.4+ KB


# 探索性分析

In [6]:
df_cor = df.drop(columns='性別').corr()
df_cor

,年齡,收入（千）,消費指數（1~100）
年齡,1.000000,0.031519,-0.127454
收入（千）,0.031519,1.000000,0.031476
消費指數（1~100）,-0.127454,0.031476,1.000000


類別轉換

In [7]:
df['性別'].unique()

array(['女', '男'], dtype=object)

LabelEncoder 主要用在目標變數（標籤），把文字標籤轉成數字。

OrdinalEncoder 則是用在特徵欄位，把分類特徵轉成數字。

In [8]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder(categories=[['男','女']])
df[['性別']] = encoder.fit_transform(df[['性別']])
df


,性別,年齡,收入（千）,消費指數（1~100）
0,1.0,74,38,81
1,1.0,51,71,91
2,1.0,30,65,10
3,1.0,88,49,17
4,1.0,55,48,70
...,...,...,...,...
195,0.0,86,84,82
196,0.0,59,52,30
197,1.0,63,29,61
198,0.0,67,80,9


# 模型訓練


In [9]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)#將客戶分成3群
kmeans.fit(df)

KMeans(n_clusters=3)

In [10]:
# 為每筆資料加上分群結果
df['cluster'] = kmeans.labels_
df

,性別,年齡,收入（千）,消費指數（1~100）,cluster
0,1.0,74,38,81,1
1,1.0,51,71,91,1
2,1.0,30,65,10,2
3,1.0,88,49,17,2
4,1.0,55,48,70,1
...,...,...,...,...,...
195,0.0,86,84,82,1
196,0.0,59,52,30,2
197,1.0,63,29,61,1
198,0.0,67,80,9,0


# 模型評估

In [11]:
# 查看各群特質
#衡量群與群之間是否分得夠開，以及每一群內是否夠緊密
#分數越高，代表分群效果越好
df.groupby('cluster').mean()


,性別,年齡,收入（千）,消費指數（1~100）
cluster,,,,
0,0.454545,56.090909,84.659091,33.977273
1,0.442308,49.432692,55.490385,76.576923
2,0.384615,56.788462,43.634615,20.250000


In [12]:
# 評估分群效果
from sklearn.metrics import calinski_harabasz_score
score = calinski_harabasz_score(df, kmeans.labels_)
score

np.float64(82.78818624599486)

# 調整模型


In [13]:
df_nocluster=df.drop(columns='cluster')
for i in range(2,16):
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(df_nocluster)
  score = calinski_harabasz_score(df_nocluster, kmeans.labels_)
  print(f'k={i} score={score}')

k=2 score=111.68583137120146
k=3 score=96.52156233412978
k=4 score=90.29850192450891
k=5 score=90.66644361687116
k=6 score=91.85989659722046
k=7 score=91.4263265797782
k=8 score=93.93161524025672
k=9 score=97.53588561343925
k=10 score=93.1831037017147
k=11 score=87.56626743580267
k=12 score=86.76845944452073
k=13 score=90.03292578295674
k=14 score=85.66681990781024
k=15 score=82.63127055616668


In [17]:
from sklearn.cluster import KMeans

# 移除 cluster 欄並建立 k=2 模型
df_nocluster = df.drop(columns='cluster', errors='ignore')  # 避免 cluster 不存在會報錯
kmeans = KMeans(n_clusters=2)
kmeans.fit(df_nocluster)

# 加上分群結果
df['cluster_2'] = kmeans.labels_

In [18]:
# 查看各群特質
#衡量群與群之間是否分得夠開，以及每一群內是否夠緊密
#分數越高，代表分群效果越好
df.groupby('cluster').mean()

,性別,年齡,收入（千）,消費指數（1~100）,cluster_2
cluster,,,,,
0,0.454545,56.090909,84.659091,33.977273,0.795455
1,0.442308,49.432692,55.490385,76.576923,0.000000
2,0.384615,56.788462,43.634615,20.250000,0.923077
